In [1]:
import math


FILE_PATH = "data/382_weather_prediction/01"


def rmse(y_true, y_pred):
    s = 0.0
    for a, b in zip(y_true, y_pred):
        s += (a - b) ** 2
    return math.sqrt(s / len(y_true))


def mean(xs):
    return sum(xs) / len(xs)


def load_series(path):
    with open(path, "r", encoding="utf-8") as f:
        data = []
        for line in f:
            for x in line.split():
                data.append(float(x))
    return data


def simulate(series, train_size, w1, w2, w3, ma_window=7):
    train = series[:train_size]
    test = series[train_size:]

    doy_sum = [0.0] * 365
    doy_cnt = [0] * 365
    for i, v in enumerate(train):
        d = i % 365
        doy_sum[d] += v
        doy_cnt[d] += 1

    global_mean = mean(train)
    doy_mean = [
        doy_sum[i] / doy_cnt[i] if doy_cnt[i] else global_mean
        for i in range(365)
    ]

    history = train[:]
    preds = []

    for real in test:
        last = history[-1]
        ma = mean(history[-ma_window:])
        d = len(history) % 365
        seasonal = doy_mean[d]

        pred = w1 * last + w2 * ma + w3 * seasonal
        preds.append(pred)

        history.append(real)
        doy_sum[d] += real
        doy_cnt[d] += 1
        doy_mean[d] = doy_sum[d] / doy_cnt[d]

    return rmse(test, preds)


def find_best_weights(series, train_size):
    best = (float("inf"), None)

    for w1 in [x / 100 for x in range(80, 101, 2)]:
        for w2 in [x / 100 for x in range(0, 21, 2)]:
            w3 = 1.0 - w1 - w2
            if w3 < 0:
                continue
            r = simulate(series, train_size, w1, w2, w3)
            if r < best[0]:
                best = (r, (w1, w2, w3))

    return best


def main():
    series = load_series(FILE_PATH)
    train_size = int(len(series) * 0.7)

    base_preds = series[train_size - 1 : -1]
    base_rmse = rmse(series[train_size:], base_preds)

    best_rmse, (w1, w2, w3) = find_best_weights(series, train_size)
    model_rmse = simulate(series, train_size, w1, w2, w3)

    print(f"baseline_rmse = {base_rmse:.6f}")
    print(f"model_rmse    = {model_rmse:.6f}")
    print(f"delta         = {base_rmse - model_rmse:.6f}")
    print(f"weights       = w1={w1:.2f}, w2={w2:.2f}, w3={w3:.2f}")


if __name__ == "__main__":
    main()

baseline_rmse = 6.395666
model_rmse    = 6.018102
delta         = 0.377564
weights       = w1=0.80, w2=0.18, w3=0.02


In [ ]:
import sys


def mean(xs):
    return sum(xs) / len(xs)


def main():
    n = int(sys.stdin.readline())
    history = [float(sys.stdin.readline()) for _ in range(n)]
    m = int(sys.stdin.readline())

    doy_sum = [0.0] * 365
    doy_cnt = [0] * 365

    for i, v in enumerate(history):
        d = i % 365
        doy_sum[d] += v
        doy_cnt[d] += 1

    global_mean = mean(history)
    doy_mean = [
        doy_sum[i] / doy_cnt[i] if doy_cnt[i] else global_mean
        for i in range(365)
    ]

    w1, w2, w3 = 0.80, 0.18, 0.02
    ma_window = 7

    for _ in range(m):
        last = history[-1]
        ma = mean(history[-ma_window:])
        d = len(history) % 365
        seasonal = doy_mean[d]

        pred = w1 * last + w2 * ma + w3 * seasonal

        print(f"{pred:.6f}")
        sys.stdout.flush()

        real = float(sys.stdin.readline())
        history.append(real)

        doy_sum[d] += real
        doy_cnt[d] += 1
        doy_mean[d] = doy_sum[d] / doy_cnt[d]


if __name__ == "__main__":
    main()
